In [2]:
import pandas as pd

call_samples = pd.read_csv("plink.imiss", sep="\s+")
related_sample = pd.read_csv("pihat_min0.2.genome", sep="\s+")

In [3]:
related_sample

,FID1,IID1,FID2,IID2,RT,EZ,Z0,Z1,Z2,PI_HAT,PHE,DST,PPC,RATIO
0,1377,NA11891,1377,NA10865,PO,0.5,0.0013,0.9976,0.0010,0.4998,0,0.823890,1.0,3614.0000
1,1349,NA11843,1349,NA10853,PO,0.5,0.0021,0.9906,0.0073,0.5026,-1,0.824903,1.0,926.5000
2,1330,NA12341,1330,NA12335,PO,0.5,0.0000,1.0000,0.0000,0.5000,0,0.822882,1.0,924.7500
3,1444,NA12739,1444,NA12749,PO,0.5,0.0018,0.9943,0.0040,0.5011,-1,0.824364,1.0,NaN
4,1444,NA12739,1444,NA12748,PO,0.5,0.0015,0.9950,0.0035,0.5010,0,0.824316,1.0,1227.6667
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
92,1459,NA12864,1459,NA12872,PO,0.5,0.0000,1.0000,0.0000,0.5000,0,0.822393,1.0,1851.5000
93,1459,NA12864,1459,NA12873,PO,0.5,0.0000,1.0000,0.0000,0.5000,-1,0.822699,1.0,916.7500
94,1444,NA12751,1444,NA12740,PO,0.5,0.0017,0.9939,0.0043,0.5013,0,0.824431,1.0,1839.0000
95,1362,NA10861,1362,NA11994,PO,0.5,0.0009,0.9942,0.0048,0.5020,0,0.824601,1.0,1859.5000


In [4]:
call_samples

,FID,IID,MISS_PHENO,N_MISS,N_GENO,F_MISS
0,1328,NA06989,N,2181,1073226,0.002032
1,1377,NA11891,N,13586,1073226,0.012660
2,1349,NA11843,N,813,1073226,0.000758
3,1330,NA12341,N,3697,1073226,0.003445
4,1328,NA06984,N,6478,1073226,0.006036
...,...,...,...,...,...,...
105,1362,NA11994,N,509,1073226,0.000474
106,1459,NA12873,N,3978,1073226,0.003707
107,1454,NA12815,N,436,1073226,0.000406
108,1346,NA12043,N,3041,1073226,0.002834


In [ ]:
import networkx as nx

G = nx.from_pandas_edgelist(related_sample, 'IID1', 'IID2')

groups = list(nx.connected_components(G))
F_dict = dict(call_samples[["IID", "F_MISS"]].values)

# Make sure sample contains in miss file
filter_groups = []
for group in groups:
    correct_group = []
    for sample in group:
        if F_dict.get(sample):
            correct_group.append(sample)
    filter_groups.append(correct_group)

# Find samples should be drop (max F_MISS in group)
drop_samples = []
for group in filter_groups:
    scores = list(map(lambda sample: F_dict[sample], group))
    drop_sample = group[scores.index(max(scores))]
    drop_samples.append(drop_sample)

In [17]:
drop_sample_df = pd.DataFrame(data={"groups": filter_groups, "sample_max_score": drop_samples})
drop_sample_df = drop_sample_df[drop_sample_df["groups"].map(lambda x: len(x) != 1)]
drop_sample_df

,groups,sample_max_score
0,"[NA11892, NA11891]",NA11891
2,"[NA12340, NA12341]",NA12341
3,"[NA12749, NA12748]",NA12749
4,"[NA12890, NA12889]",NA12889
6,"[NA07045, NA12813, NA12812, NA06986]",NA07045
7,"[NA12272, NA12273]",NA12272
8,"[NA11930, NA11931]",NA11931
10,"[NA07051, NA07031]",NA07031
11,"[NA12399, NA12400]",NA12400
12,"[NA12348, NA12347]",NA12348
